# Imports

In [364]:
import pandas as pd
import numpy as np
import nltk
import random
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Load Data

In [384]:
pd.set_option('display.max_colwidth', None)

In [385]:
albums = pd.read_csv('nlp_albums.csv', index_col=0)

In [386]:
df = albums.drop(columns=['ars_name', 'rel_date', 'gens', 'descs', 'avg_rat', 'album', 'genre'])

In [387]:
indices = pd.Series(df.index, index=df['name'])

# Modeling

## Linear Kernel with Tfidf Vectorizer

In [388]:
tfidf = TfidfVectorizer()

In [389]:
matrix = tfidf.fit_transform(df['text'])

In [390]:
matrix_weight = tfidf.fit_transform(df['weight_text'])

In [391]:
cosine_sim = linear_kernel(matrix, matrix)

In [394]:
def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:11]
    sim_index = [x[0] for x in sim_scores]
    return pd.DataFrame({'name':albums['name'].iloc[sim_index], 
                         'genres':albums['gens'].iloc[sim_index], 
                         'descriptions':albums['descs'].iloc[sim_index]})

In [395]:
get_recommendations('Bob Dylan - Blonde on Blonde')

,name,genres,descriptions
42,Bob Dylan - Blonde on Blonde,"Folk Rock, Singer/Songwriter","poetic, surreal, cryptic, sarcastic, bittersweet, romantic, introspective, male vocals, playful, conscious"
33,Bob Dylan - Highway 61 Revisited,"Folk Rock, Singer/Songwriter","poetic, cryptic, surreal, sarcastic, male vocals, passionate, conscious, rebellious, philosophical, introspective"
1503,Nacho Vegas - Desaparezca aquí,"Folk Rock, Singer/Songwriter","poetic, introspective, male vocals"
78,Bob Dylan - Bringing It All Back Home,"Folk Rock, Singer/Songwriter, Contemporary Folk","poetic, sarcastic, satirical, male vocals, surreal, conscious, political, cryptic, protest, humorous"
425,Bob Dylan - Desire,"Folk Rock, Singer/Songwriter","male vocals, poetic, bittersweet, passionate, history, female vocals, political, warm, conscious, romantic"
2177,Van Morrison - Into the Music,"Singer/Songwriter, Folk Rock","introspective, passionate, romantic, love, male vocals, melodic"
1470,Nacho Vegas - Cajas de música difíciles de parar,"Folk Rock, Singer/Songwriter",male vocals
3528,Richard Thompson - Rumor and Sigh,"Folk Rock, Singer/Songwriter",male vocals
1015,Tim Buckley - Happy Sad,"Singer/Songwriter, Psychedelic Folk","happy, sad, introspective, romantic, bittersweet, male vocals, poetic, mellow, passionate, warm"
2689,Smog - Knock Knock,Singer/Songwriter,"pastoral, introspective, cryptic, mellow, male vocals, breakup, sarcastic, bittersweet, sentimental, warm"


In [396]:
get_recommendations('The Beatles - Abbey Road')

,name,genres,descriptions
7,The Beatles - Abbey Road,Pop Rock,"melodic, warm, male vocals, bittersweet, summer, uplifting, love, romantic, medley, happy"
1088,The Velvet Underground - Loaded,Pop Rock,"melodic, playful, male vocals, summer, bittersweet, warm, humorous, sentimental, uplifting, love"
1684,The Beatles - Help!,Pop Rock,"bittersweet, melodic, love, male vocals, romantic, uplifting, longing, energetic"
598,Paul McCartney & Wings - Band on the Run,Pop Rock,"melodic, happy, male vocals, uplifting, playful"
71,The Beatles - Rubber Soul,Pop Rock,"melodic, love, male vocals, bittersweet, romantic, mellow, autumn, melancholic, eclectic, happy"
3083,Jason Falkner - Presents Author Unknown,"Power Pop, Pop Rock","uplifting, melodic, male vocals, playful, summer, love"
960,Big Star - #1 Record,"Power Pop, Folk Pop","melodic, romantic, bittersweet, uplifting, summer, playful, optimistic, love, male vocals, sentimental"
3324,Boston - Boston,"Hard Rock, AOR","energetic, melodic, male vocals, uplifting, happy, anthemic, love, romantic, warm, bittersweet"
913,Jorge Ben - Samba esquema novo,Samba,"uplifting, warm, tropical, summer, melodic, playful, mellow, happy, romantic, male vocals"
3948,Teenage Fanclub - Bandwagonesque,"Power Pop, Jangle Pop","melodic, male vocals, bittersweet, romantic, love, sentimental, energetic, summer, warm"


In [397]:
get_recommendations('Metallica - Master of Puppets')

,name,genres,descriptions
91,Metallica - Master of Puppets,Thrash Metal,"aggressive, heavy, angry, energetic, male vocals, rebellious, political, serious, uncommon time signatures, progressive"
458,Metallica - ...And Justice for All,Thrash Metal,"political, angry, technical, heavy, male vocals, energetic, progressive, aggressive, dark, serious"
1970,Аспид [Aspid] - Кровоизлияние,Technical Thrash Metal,"complex, uncommon time signatures, energetic, technical, aggressive, male vocals, angry, heavy, political"
1916,Anthrax - Among the Living,Thrash Metal,"energetic, rhythmic, male vocals, aggressive, political, heavy, melodic, angry, rebellious, conscious"
879,Coroner - Mental Vortex,Technical Thrash Metal,"technical, uncommon time signatures, heavy, male vocals, aggressive, philosophical, complex, angry, progressive, dark"
1383,Overkill - Horrorscope,Thrash Metal,"male vocals, rebellious, heavy, dark, rhythmic, energetic, angry"
1351,Coroner - Punishment for Decadence,"Technical Thrash Metal, Thrash Metal","technical, aggressive, energetic, heavy, male vocals, complex, uncommon time signatures, death, apocalyptic, chaotic"
2967,Forbidden - Twisted Into Form,Thrash Metal,"energetic, technical, heavy, male vocals, progressive, angry"
698,Artillery - By Inheritance,Thrash Metal,"energetic, technical, melodic, political, angry, male vocals, aggressive, pessimistic, heavy"
2921,Between the Buried and Me - The Great Misdirect,Progressive Metal,"male vocals, technical, progressive, complex, heavy, uncommon time signatures, eclectic, energetic, dense, aggressive"


In [398]:
get_recommendations('Jay-Z - The Blueprint')

,name,genres,descriptions
612,Jay-Z - The Blueprint,"East Coast Hip Hop, Chipmunk Soul","boastful, male vocals, rhythmic, sampling, urban, triumphant, crime, introspective, lush, sentimental"
1189,Jay-Z - The Black Album,East Coast Hip Hop,"boastful, male vocals, rhythmic, urban, crime, drugs, bittersweet, conscious, introspective, sampling"
1786,Nas - The Lost Tapes,"East Coast Hip Hop, Conscious Hip Hop, Hardcore Hip Hop","urban, sampling, male vocals, conscious, philosophical, introspective, crime, rhythmic, sentimental, boastful"
4391,Cormega - The True Meaning,"East Coast Hip Hop, Hardcore Hip Hop","introspective, rhythmic, sampling, urban, boastful, male vocals, conscious"
2939,Kool G Rap & DJ Polo - Wanted: Dead or Alive,"East Coast Hip Hop, Hardcore Hip Hop","rhythmic, urban, sampling, boastful, male vocals, energetic, crime, conscious, dark, sexual"
252,Beastie Boys - Paul's Boutique,East Coast Hip Hop,"sampling, energetic, humorous, urban, playful, sexual, male vocals, crime, boastful, complex"
1327,Fugees - The Score,"East Coast Hip Hop, Conscious Hip Hop","political, crime, female vocals, conscious, male vocals, lush, boastful, introspective, nocturnal, sampling"
20,Nas - Illmatic,"East Coast Hip Hop, Boom Bap, Hardcore Hip Hop","urban, crime, introspective, sampling, philosophical, conscious, rhythmic, drugs, male vocals, boastful"
2620,Lords of the Underground - Here Come the Lords,"East Coast Hip Hop, Boom Bap, Hardcore Hip Hop","boastful, rhythmic, dark, urban, aggressive, male vocals, energetic, sampling, crime"
1760,Masta Ace - A Long Hot Summer,"East Coast Hip Hop, Boom Bap","conscious, urban, concept album, summer, rhythmic, sampling, crime, male vocals, introspective, boastful"


In [399]:
get_recommendations('Ramones - Ramones')

,name,genres,descriptions
793,Ramones - Ramones,Punk Rock,"energetic, raw, urban, rebellious, male vocals, melodic, repetitive, playful, minimalistic, violence"
471,Ramones - Rocket to Russia,Punk Rock,"energetic, melodic, playful, rebellious, male vocals, humorous, urban, anthemic, repetitive, raw"
1546,The Saints - (I'm) Stranded,Punk Rock,"energetic, raw, male vocals, rebellious, sexual, playful, urban"
2333,Ramones - Road to Ruin,Punk Rock,"rebellious, male vocals, energetic, repetitive, playful, melodic, humorous, anthemic, raw"
229,Wire - Pink Flag,"Punk Rock, Art Punk","energetic, minimalistic, male vocals, raw, rebellious, sarcastic, cryptic, abstract, playful, quirky"
4222,The Boys - The Boys,Punk Rock,"male vocals, melodic, energetic, rebellious"
4058,The Lawrence Arms - Oh! Calcutta!,Punk Rock,"male vocals, anxious, energetic, raw, rebellious"
1488,Radio Birdman - Radios Appear,Punk Rock,"urban, male vocals, energetic"
2449,Buzzcocks - Another Music in a Different Kitchen,Punk Rock,"energetic, male vocals, melodic, raw, love, rebellious, sexual"
1240,Ramones - Leave Home,Punk Rock,"energetic, melodic, playful, male vocals, raw, sarcastic, humorous, anthemic, repetitive"


In [400]:
get_recommendations('Bob Marley and The Wailers - Catch a Fire')

,name,genres,descriptions
502,Bob Marley and The Wailers - Catch a Fire,Roots Reggae,"mellow, rhythmic, conscious, melodic, summer, male vocals, tropical, warm, passionate, uplifting"
344,Bob Marley & The Wailers - Exodus,Roots Reggae,"uplifting, male vocals, warm, optimistic, political, rhythmic, tropical, summer, melodic, mellow"
2118,Bob Marley & The Wailers - Kaya,Roots Reggae,"warm, love, male vocals, mellow, soothing, tropical, peaceful, summer, melodic, rhythmic"
2431,Bob Marley and The Wailers - Soul Revolution,Roots Reggae,"male vocals, melodic, rebellious, warm, mellow, rhythmic, conscious, tropical, drugs, bittersweet"
1596,Culture - Two Sevens Clash,Roots Reggae,"spiritual, male vocals, rhythmic, optimistic, uplifting, warm, tropical, conscious, hypnotic, passionate"
1577,Black Uhuru - Showcase,Roots Reggae,"conscious, mellow, male vocals, rhythmic, female vocals, hypnotic, urban, religious, summer, tropical"
2480,Toots & The Maytals - In the Dark,Roots Reggae,"mellow, warm, melodic, summer, bittersweet, male vocals"
3291,Bob Marley & The Wailers - Survival,Roots Reggae,"political, warm, uplifting, conscious"
4392,Dennis Brown - Visions of Dennis Brown,Roots Reggae,"male vocals, rhythmic, soothing, calm, conscious, melodic, political, tropical"
503,The Congos - Heart of The Congos,Roots Reggae,"mellow, warm, religious, spiritual, rhythmic, tropical, psychedelic, lo-fi, summer, melodic"


## Non-negative Matrix Factorization with Tfidf Vectorizer

In [402]:
def find_similar_albums(target_title, top_k=10):
    random.seed(42)
    
    model = NMF(n_components=50, max_iter=1000) 
    latent_factors = model.fit_transform(matrix)
    target_index = df[df['name'] == target_title].index[0]
    similarities = cosine_similarity(latent_factors[target_index].reshape(1, -1), latent_factors)
    sim_index = similarities.argsort()[0][::-1][:top_k +1]
    return pd.DataFrame({'name':albums['name'].iloc[sim_index], 
                         'genres':albums['gens'].iloc[sim_index], 
                         'descriptions':albums['descs'].iloc[sim_index]})

In [403]:
find_similar_albums('Bob Dylan - Blonde on Blonde')

,name,genres,descriptions
42,Bob Dylan - Blonde on Blonde,"Folk Rock, Singer/Songwriter","poetic, surreal, cryptic, sarcastic, bittersweet, romantic, introspective, male vocals, playful, conscious"
33,Bob Dylan - Highway 61 Revisited,"Folk Rock, Singer/Songwriter","poetic, cryptic, surreal, sarcastic, male vocals, passionate, conscious, rebellious, philosophical, introspective"
2689,Smog - Knock Knock,Singer/Songwriter,"pastoral, introspective, cryptic, mellow, male vocals, breakup, sarcastic, bittersweet, sentimental, warm"
78,Bob Dylan - Bringing It All Back Home,"Folk Rock, Singer/Songwriter, Contemporary Folk","poetic, sarcastic, satirical, male vocals, surreal, conscious, political, cryptic, protest, humorous"
3,Radiohead - Kid A,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, anxious, cryptic, sombre, abstract, introspective, male vocals"
3544,R.A.P. Ferreira - Purple Moonlight Pages,"Jazz Rap, Abstract Hip Hop","poetic, introspective, male vocals, abstract, philosophical, sampling, rhythmic, optimistic, mellow, cryptic"
1203,Bob Dylan - John Wesley Harding,"Folk Rock, Singer/Songwriter, Contemporary Folk","poetic, pastoral, acoustic, male vocals, cryptic, history, conscious, melodic, religious, sparse"
345,Robert Wyatt - Rock Bottom,"Canterbury Scene, Art Rock","sombre, atmospheric, surreal, poetic, melancholic, cryptic, male vocals, romantic, mysterious, progressive"
3205,Гражданская оборона [Grazhdanskaya Oborona] - Лунный переворот,"Psychedelic Rock, Post-Punk","psychedelic, passionate, cryptic, existential, anthemic, Wall of Sound, surreal, philosophical, poetic, mysterious"
1283,Pavement - Brighten the Corners,Indie Rock,"mellow, melodic, male vocals, abstract, bittersweet, playful, autumn, humorous, cryptic, surreal"


In [404]:
find_similar_albums('The Beatles - Abbey Road')

,name,genres,descriptions
7,The Beatles - Abbey Road,Pop Rock,"melodic, warm, male vocals, bittersweet, summer, uplifting, love, romantic, medley, happy"
2303,山下達郎 [Tatsuro Yamashita] - For You,City Pop,"summer, uplifting, passionate, warm, romantic, melodic, optimistic, male vocals, love, lush"
2408,Tats Yamashita - Big Wave,"City Pop, Film Soundtrack","summer, melodic, lush, happy, warm, love, Wall of Sound, uplifting, romantic, sentimental"
955,Big Star - Radio City,Power Pop,"melodic, bittersweet, male vocals, romantic, warm, playful, summer, passionate, urban"
431,Stevie Wonder - Talking Book,Soul,"love, passionate, male vocals, melodic, warm, uplifting, mellow, optimistic, romantic, lush"
1358,Todd Rundgren - Something / Anything?,Pop Rock,"melodic, romantic, eclectic, humorous, playful, male vocals, love, bittersweet, uplifting, optimistic"
223,Jorge Ben - A Tábua de Esmeralda,"MPB, Samba-rock","warm, tropical, uplifting, summer, male vocals, playful, lush, occult, happy, romantic"
913,Jorge Ben - Samba esquema novo,Samba,"uplifting, warm, tropical, summer, melodic, playful, mellow, happy, romantic, male vocals"
672,Michael Jackson - Off the Wall,"Disco, Pop","party, happy, male vocals, warm, rhythmic, love, energetic, lush, nocturnal, romantic"
960,Big Star - #1 Record,"Power Pop, Folk Pop","melodic, romantic, bittersweet, uplifting, summer, playful, optimistic, love, male vocals, sentimental"


In [405]:
find_similar_albums('Metallica - Master of Puppets')

,name,genres,descriptions
91,Metallica - Master of Puppets,Thrash Metal,"aggressive, heavy, angry, energetic, male vocals, rebellious, political, serious, uncommon time signatures, progressive"
2797,Death Toll 80k - Harsh Realities,Grindcore,"heavy, aggressive, energetic, male vocals, raw, dense, apocalyptic, political, hateful, angry"
4215,Snapcase - Progression Through Unlearning,Metalcore,"aggressive, heavy, energetic, angry, male vocals, raw, uplifting, uncommon time signatures, rhythmic"
1393,Terrorizer - World Downfall,Deathgrind,"aggressive, political, male vocals, raw, heavy, energetic, conscious, apocalyptic, misanthropic, angry"
4394,Nails - Abandon All Life,"Grindcore, Powerviolence","aggressive, heavy, chaotic, angry, misanthropic, rebellious, violence, infernal, passionate, male vocals"
2710,Brutal Truth - Extreme Conditions Demand Extreme Responses,"Deathgrind, Grindcore","angry, political, energetic, male vocals, aggressive, heavy, anti-religious, apocalyptic, dark, raw"
2656,Assück - Misery Index,Deathgrind,"aggressive, heavy, pessimistic, male vocals, anarchism, political, raw, nihilistic, angry, alienation"
2299,Eyehategod - Dopesick,Sludge Metal,"drugs, heavy, angry, aggressive, hateful, misanthropic, nihilistic, raw, male vocals, pessimistic"
3963,Ratos de Porão - Brasil,Crossover Thrash,"protest, aggressive, angry, energetic, conscious, political, rebellious, heavy, male vocals, alienation"
1141,Converge - Axe to Fall,Metalcore,"aggressive, energetic, passionate, angry, dark, male vocals, heavy, noisy, chaotic, dense"


In [406]:
find_similar_albums('Jay-Z - The Blueprint')

,name,genres,descriptions
612,Jay-Z - The Blueprint,"East Coast Hip Hop, Chipmunk Soul","boastful, male vocals, rhythmic, sampling, urban, triumphant, crime, introspective, lush, sentimental"
4391,Cormega - The True Meaning,"East Coast Hip Hop, Hardcore Hip Hop","introspective, rhythmic, sampling, urban, boastful, male vocals, conscious"
20,Nas - Illmatic,"East Coast Hip Hop, Boom Bap, Hardcore Hip Hop","urban, crime, introspective, sampling, philosophical, conscious, rhythmic, drugs, male vocals, boastful"
1189,Jay-Z - The Black Album,East Coast Hip Hop,"boastful, male vocals, rhythmic, urban, crime, drugs, bittersweet, conscious, introspective, sampling"
3859,Big K.R.I.T. - K.R.I.T Wuz Here,"Southern Hip Hop, Dirty South, Hardcore Hip Hop","male vocals, conscious, sampling, introspective, boastful, warm, rhythmic, energetic"
2350,Scarface - The Fix,"Southern Hip Hop, Gangsta Rap","introspective, crime, rhythmic, violence, religious, sombre, male vocals, conscious"
414,Jay-Z - Reasonable Doubt,"East Coast Hip Hop, Boom Bap, Gangsta Rap","crime, boastful, urban, sombre, drugs, male vocals, serious, introspective, rhythmic, sampling"
1786,Nas - The Lost Tapes,"East Coast Hip Hop, Conscious Hip Hop, Hardcore Hip Hop","urban, sampling, male vocals, conscious, philosophical, introspective, crime, rhythmic, sentimental, boastful"
2500,Pezet-Noon - Muzyka poważna,"Boom Bap, Hip Hop","urban, rhythmic, introspective, sampling, male vocals, passionate, dense, melancholic, conscious"
2460,Pezet-Noon - Muzyka klasyczna,"Boom Bap, Hip Hop","sampling, male vocals, rhythmic, urban, dense, melancholic, passionate, introspective, conscious"


In [407]:
find_similar_albums('Ramones - Ramones')

,name,genres,descriptions
793,Ramones - Ramones,Punk Rock,"energetic, raw, urban, rebellious, male vocals, melodic, repetitive, playful, minimalistic, violence"
471,Ramones - Rocket to Russia,Punk Rock,"energetic, melodic, playful, rebellious, male vocals, humorous, urban, anthemic, repetitive, raw"
2333,Ramones - Road to Ruin,Punk Rock,"rebellious, male vocals, energetic, repetitive, playful, melodic, humorous, anthemic, raw"
763,The Fall - Hex Enduction Hour,"Post-Punk, Art Punk","sarcastic, repetitive, misanthropic, energetic, cryptic, raw, noisy, rebellious, male vocals, satirical"
1240,Ramones - Leave Home,Punk Rock,"energetic, melodic, playful, male vocals, raw, sarcastic, humorous, anthemic, repetitive"
876,Suicide - Suicide,"Synth Punk, Minimal Synth","anxious, dark, mechanical, repetitive, disturbing, minimalistic, ominous, male vocals, raw, misanthropic"
1312,Fela and The Africa 70 - Shakara,Afrobeat,"energetic, rhythmic, conscious, passionate, sarcastic, playful, repetitive, hypnotic, warm, rebellious"
2041,Playboi Carti - Die Lit,"Trap, Cloud Rap, Southern Hip Hop","repetitive, hypnotic, male vocals, boastful, drugs, sexual, hedonistic, rhythmic, ethereal, party"
1889,Melvins - Lysol,"Sludge Metal, Drone Metal","heavy, hypnotic, repetitive, male vocals, atmospheric, apocalyptic, rebellious, ominous, raw, aggressive"
1623,The Fall - Grotesque (After the Gramme),Post-Punk,"sarcastic, satirical, cryptic, raw, energetic, misanthropic, urban, repetitive, conscious, manic"


In [408]:
find_similar_albums('Bob Marley and The Wailers - Catch a Fire')

,name,genres,descriptions
502,Bob Marley and The Wailers - Catch a Fire,Roots Reggae,"mellow, rhythmic, conscious, melodic, summer, male vocals, tropical, warm, passionate, uplifting"
344,Bob Marley & The Wailers - Exodus,Roots Reggae,"uplifting, male vocals, warm, optimistic, political, rhythmic, tropical, summer, melodic, mellow"
1409,Toots & The Maytals - Funky Kingston,"Reggae, Rocksteady","tropical, warm, energetic, optimistic, rhythmic, uplifting, melodic, passionate, raw, summer"
3079,Ini Kamoze - Ini Kamoze,"Dancehall, Reggae","rhythmic, warm, summer, tropical, male vocals, playful, melodic, uplifting, urban, psychedelic"
3122,Fẹla Ransome-Kuti and The Africa 70 - He Miss Road,Afrobeat,"male vocals, rhythmic, psychedelic, energetic, political, uplifting, warm, happy"
420,Jorge Ben - África Brasil,"Samba-rock, MPB, Funk","energetic, rhythmic, sports, tropical, summer, uplifting, male vocals, passionate, warm, psychedelic"
685,Buena Vista Social Club - Buena Vista Social Club,"Son cubano, Bolero","warm, tropical, melodic, mellow, rhythmic, male vocals, acoustic, uplifting, sentimental, playful"
1757,Gilberto Gil - Gilberto Gil,Tropicália,"psychedelic, energetic, tropical, playful, male vocals, summer, uplifting, mellow, happy, rhythmic"
2708,Bob Marley & The Wailers - Rastaman Vibration,Roots Reggae,"male vocals, warm"
403,Santana - Abraxas,Latin Rock,"warm, rhythmic, psychedelic, passionate, male vocals, uplifting, summer, atmospheric, tropical, sensual"


## Cosine Similarity with Word2Vec

In [313]:
album_descriptions = list(df['text'].str.split())

In [314]:
model = Word2Vec(album_descriptions, vector_size=100, window=5, min_count=1, workers=4)

In [315]:
album_vectors = [model.wv[desc].mean(axis=0) for desc in album_descriptions]

In [318]:
def get_recommendations_w2v(title, model=model):
    query_index = df[df['name'] == title].index[0]
    query_vector = album_vectors[query_index].reshape(1, -1)
    similarity_scores = cosine_similarity(query_vector, album_vectors)
    sim_scores = list(enumerate(similarity_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_index = [x[0] for x in sim_scores if x[0] != query_index][:10] 
    return pd.DataFrame({'name':albums['name'].iloc[sim_index], 
                         'genres':albums['gens'].iloc[sim_index], 
                         'descriptions':albums['descs'].iloc[sim_index]})

In [308]:
get_recommendations_w2v('Bob Dylan - Blonde on Blonde')

,name,genres,descriptions
3693,Magnetic Fields - Holiday,"Indie Pop, Synthpop","melodic, romantic, bittersweet, sarcastic, male vocals, humorous, lo-fi, tropical, quirky, LGBT"
3203,"Raul Seixas - Krig-ha, bandolo!","Singer/Songwriter, Folk Rock, MPB","male vocals, existential, energetic, protest, passionate, philosophical, warm, poetic, playful, urban"
2311,Raul Seixas - Gita,"Singer/Songwriter, Rock, Folk Rock","sarcastic, playful, melodic, eclectic, male vocals, love, poetic, urban, protest, philosophical"
3725,Randy Newman - Bad Love,"Singer/Songwriter, Pop Rock, Piano Rock","male vocals, political, melodic, lyrics, sarcastic, ensemble, female vocals, vocal group, warm, humorous"
1478,Millie Jackson - Caught Up,"Southern Soul, Deep Soul","concept album, longing, passionate, female vocals, melodic, angry, humorous, ballad, bittersweet"
2196,Guided by Voices - Under the Bushes Under the Stars,"Indie Rock, Lo-Fi / Slacker Rock","melodic, raw, lo-fi, male vocals, uplifting, cryptic, passionate, summer, alcohol, bittersweet"
4006,Billy Bragg - Talking With the Taxman About Poetry,"Folk Rock, Singer/Songwriter","political, male vocals, conscious, protest, sentimental, energetic, love, melancholic, sarcastic"
1185,Tom Waits - Small Change,"Singer/Songwriter, Piano Blues","bittersweet, alcohol, poetic, romantic, humorous, urban, passionate, male vocals, nocturnal, playful"
4130,Donald Fagen - The Nightfly,"Sophisti-Pop, Jazz Pop, Pop Rock","lush, concept album, melodic, mellow, male vocals, romantic, sarcastic, urban, nocturnal, bittersweet"
3516,Dick Gaughan - Handful of Earth,Scottish Folk Music,"acoustic, folklore, political, pastoral, male vocals, technique, passionate"


In [321]:
get_recommendations_w2v('The Beatles - Abbey Road')

,name,genres,descriptions
2295,Teenage Fanclub - Grand Prix,"Power Pop, Jangle Pop","melodic, bittersweet, male vocals, longing, uplifting, romantic, sentimental, love, mellow, warm"
4401,Minnie Riperton - Adventures in Paradise,"Smooth Soul, Pop Soul","lush, female vocals, romantic, uplifting, summer, love, bittersweet, passionate, sensual, warm"
2747,Nancy Sinatra & Lee Hazlewood - Nancy & Lee,"Baroque Pop, Country Pop","female vocals, male vocals, pastoral, lush, melodic, warm, mellow, romantic, sentimental, love"
2038,Minnie Riperton - Come to My Garden,"Smooth Soul, Pop Soul","lush, passionate, female vocals, romantic, love, soft, uplifting, melodic, rain, longing"
4230,Japanese Breakfast - Jubilee,"Indie Pop, Chamber Pop","lush, warm, melodic, optimistic, love, female vocals, bittersweet, sentimental, playful, introspective"
2256,Elvis Presley - From Elvis in Memphis,"Blue-Eyed Soul, Country Soul","male vocals, love, passionate, melodic, breakup, sentimental, longing, bittersweet, lush, warm"
2993,Maxwell - Maxwell's Urban Hang Suite,"Neo-Soul, Contemporary R&B","sensual, mellow, melodic, sexual, love, lush, male vocals, soothing, romantic, soft"
71,The Beatles - Rubber Soul,Pop Rock,"melodic, love, male vocals, bittersweet, romantic, mellow, autumn, melancholic, eclectic, happy"
3535,St. Vincent - Actor,"Art Pop, Indie Pop","bittersweet, romantic, introspective, lush, melodic, warm, female vocals, passionate, mellow, love"
3593,The Stylistics - The Stylistics,"Philly Soul, Smooth Soul","lush, romantic, melodic, love, soft, male vocals, soothing"


In [337]:
get_recommendations_w2v('Metallica - Master of Puppets')

,name,genres,descriptions
4203,The Chariot - Long Live,Metalcore,"noisy, chaotic, aggressive, uncommon time signatures, angry, religious, male vocals, energetic, passionate, rebellious"
1537,Atheist - Piece of Time,Technical Death Metal,"technical, energetic, philosophical, complex, heavy, conscious, aggressive, uncommon time signatures, male vocals, existential"
4215,Snapcase - Progression Through Unlearning,Metalcore,"aggressive, heavy, energetic, angry, male vocals, raw, uplifting, uncommon time signatures, rhythmic"
1970,Аспид [Aspid] - Кровоизлияние,Technical Thrash Metal,"complex, uncommon time signatures, energetic, technical, aggressive, male vocals, angry, heavy, political"
1967,The Dillinger Escape Plan - One of Us Is the Killer,Mathcore,"uncommon time signatures, chaotic, energetic, technical, aggressive, angry, complex, male vocals, manic, heavy"
3325,The Dillinger Escape Plan - Miss Machine,Mathcore,"uncommon time signatures, energetic, technical, manic, angry, complex, male vocals, aggressive, chaotic, heavy"
815,Immolation - Here in After,Death Metal,"complex, technical, anti-religious, uncommon time signatures, rhythmic, dark, male vocals, infernal, heavy, aggressive"
3811,Today Is the Day - Willpower,"Noise Rock, Post-Hardcore","misanthropic, technical, heavy, uncommon time signatures, passionate, anxious, dissonant, male vocals, angry, raw"
3256,The Dillinger Escape Plan - Calculating Infinity,Mathcore,"uncommon time signatures, manic, technical, chaotic, angry, energetic, aggressive, complex, male vocals, dissonant"
1045,Botch - We Are the Romans,Mathcore,"uncommon time signatures, aggressive, heavy, energetic, technical, male vocals, chaotic, dissonant, angry, complex"


In [338]:
get_recommendations_w2v('Jay-Z - The Blueprint')

,name,genres,descriptions
261,Gang Starr - Moment of Truth,"East Coast Hip Hop, Boom Bap","urban, optimistic, male vocals, philosophical, conscious, sampling, melodic, rhythmic, uplifting, introspective"
1278,Common Sense - Resurrection,"Conscious Hip Hop, Boom Bap, Jazz Rap","urban, conscious, passionate, political, sampling, rhythmic, uplifting, male vocals, optimistic, love"
3111,Kendrick Lamar - Section.80,"West Coast Hip Hop, Conscious Hip Hop","conscious, urban, political, rhythmic, male vocals, concept album, mellow, drugs, introspective, Christian"
725,Goodie Mob - Soul Food,"Southern Hip Hop, Conscious Hip Hop","sombre, conscious, male vocals, urban, political, rhythmic, existential, spiritual, introspective, optimistic"
860,A Tribe Called Quest - People's Instinctive Travels and the Paths of Rhythm,"East Coast Hip Hop, Jazz Rap","playful, mellow, sampling, rhythmic, male vocals, conscious, urban, spiritual, poetic, passionate"
16,"Kendrick Lamar - good kid, m.A.A.d city","West Coast Hip Hop, Conscious Hip Hop","urban, crime, concept album, conscious, introspective, male vocals, passionate, existential, violence, bittersweet"
1189,Jay-Z - The Black Album,East Coast Hip Hop,"boastful, male vocals, rhythmic, urban, crime, drugs, bittersweet, conscious, introspective, sampling"
829,The Roots - Illadelph Halflife,"East Coast Hip Hop, Boom Bap","urban, mellow, male vocals, raw, nocturnal, serious, conscious, poetic, rhythmic"
4395,The Koreatown Oddity - Little Dominiques Nosebleed,"West Coast Hip Hop, Conscious Hip Hop, Jazz Rap","introspective, concept album, male vocals, sampling, playful, humorous, sentimental, bittersweet, surreal, lo-fi"
1745,Camp Lo - Uptown Saturday Night,"East Coast Hip Hop, Jazz Rap","urban, party, abstract, male vocals, mellow, playful, boastful, optimistic, hedonistic, uplifting"


In [339]:
get_recommendations_w2v('Ramones - Ramones')

,name,genres,descriptions
2701,AC/DC - Highway to Hell,Hard Rock,"energetic, sexual, male vocals, rebellious, rhythmic, raw, boastful, hedonistic, nocturnal, melodic"
3483,Fela Anikulapo Kuti & Egypt 80 - Beasts of No Nation,Afrobeat,"conscious, rhythmic, political, warm, passionate, angry, male vocals, protest, rebellious, dark"
2507,U2 - War,"Post-Punk, Alternative Rock","war, political, male vocals, passionate, protest, energetic, angry, anthemic, conscious, melodic"
471,Ramones - Rocket to Russia,Punk Rock,"energetic, melodic, playful, rebellious, male vocals, humorous, urban, anthemic, repetitive, raw"
3254,New York Dolls - New York Dolls,"Glam Rock, Proto-Punk, Glam Punk","energetic, rebellious, male vocals, playful, raw, urban, hedonistic, noisy"
87,The Clash - London Calling,"New Wave, Punk Rock","political, energetic, rebellious, eclectic, male vocals, rhythmic, passionate, conscious, urban, anthemic"
224,Fela Anikulapo Kuti and Afrika 70 - Zombie,Afrobeat,"protest, rhythmic, energetic, political, repetitive, passionate, conscious, rebellious, male vocals, war"
1992,Free - Tons of Sobs,Blues Rock,"passionate, raw, male vocals, rhythmic, sexual"
671,Chico Science & Nação Zumbi - Da lama ao caos,Manguebeat,"rhythmic, political, energetic, conscious, male vocals, protest, tropical, playful, passionate, angry"
1444,Fela Ransome-Kuti & The Africa '70 - Afrodisiac,Afrobeat,"energetic, tribal, male vocals, passionate, rhythmic, tropical, party, manic, political, rebellious"


In [340]:
get_recommendations_w2v('Bob Marley and The Wailers - Catch a Fire')

,name,genres,descriptions
344,Bob Marley & The Wailers - Exodus,Roots Reggae,"uplifting, male vocals, warm, optimistic, political, rhythmic, tropical, summer, melodic, mellow"
2821,Wilson Pickett - The Exciting Wilson Pickett,"Southern Soul, Deep Soul","male vocals, energetic, melodic, rhythmic, uplifting, playful, romantic, passionate, summer, love"
3682,O'Jays - Back Stabbers,Philly Soul,"love, optimistic, male vocals, conscious, passionate, vocal group, lush, melodic, playful, sentimental"
3731,Lee Dorsey - Yes We Can,"New Orleans R&B, Funk, Soul","happy, uplifting, male vocals, mellow, summer, conscious, melodic, playful, rhythmic, warm"
196,Curtis Mayfield - Curtis,Chicago Soul,"conscious, political, passionate, uplifting, warm, rhythmic, male vocals, lush, optimistic, sentimental"
57,Stevie Wonder - Songs in the Key of Life,Soul,"uplifting, male vocals, passionate, warm, optimistic, philosophical, happy, melodic, romantic, conscious"
1498,Nina Simone - I Put a Spell on You,"Vocal Jazz, Soul","female vocals, passionate, lush, sentimental, melodic, sensual, angry, rhythmic, melancholic, playful"
1884,Gil Scott-Heron/Brian Jackson - Winter in America,"Soul, Jazz-Funk, Soul Jazz","conscious, rhythmic, melancholic, political, mellow, longing, uplifting, passionate, male vocals, introspective"
1400,Gil Scott-Heron & Brian Jackson - Bridges,"Soul, Jazz-Funk, Funk","rhythmic, political, melodic, male vocals, conscious, uplifting, lush, mellow, warm, atmospheric"
3382,Michael Jackson - Bad,"Dance-Pop, Contemporary R&B, Pop","playful, romantic, energetic, male vocals, rhythmic, love, optimistic, uplifting, urban, passionate"
